In [77]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import pytorch_lightning as pl
from learn_KKL.luenberger_observer import LuenbergerObserver
from learn_KKL.system import RevDuffing
from learn_KKL.learner import Learner
from learn_KKL.utils import generate_mesh
from sklearn.model_selection import train_test_split

In [73]:
# Get reversed duffing as example system
system = RevDuffing()
data = generate_mesh(np.array([[-2, 2],[-2, 2]]),4000,'uniform')
train_data, val_data = train_test_split(data, test_size=0.3, shuffle=True)

Computed the smallest possible uniform grid for the given dimensions, then deleted 96 samples randomly to match the desired number of samples 4000.


In [74]:
# Initiate the Luenberger Observer
observer = LuenbergerObserver(2, 1, method="Autoencoder", wc=0.5)
observer.set_dynamics(system)
D = (observer.D).numpy()
valp,vecp = np.linalg.eig(D)
print(valp)

[-2.34249836+2.23482416j -2.34249836-2.23482416j -2.95812373+0.j        ]


In [80]:
# Train the model
learner = Learner(observer,system,train_data,val_data,method='Autoencoder')
learner.train()
trainer_options={'max_epochs': 5}
trainer = pl.Trainer(**trainer_options)
trainer.fit(learner)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.1 K
---------------------------------------------
26.1 K    Trainable params
0         Non-trainable params
26.1 K    Total params
0.104     Total estimated model params size (MB)


Results saved in in C:\Users\pchauris\stage\dir\repo\learn_observe_KKL\jupyter_notebooks\runs\Reversed_Duffing_Oscillator\Autoencoder\exp_0


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [81]:
tsim = (0, 50)
dt = 1e-2
tq, simulation = system.simulate(torch.transpose(torch.tensor([[1.],[1.]]),0,1), tsim, dt)
measurement = simulation[:, 0]

In [82]:
# Predict from measurement
estimation = observer.predict(measurement, tsim, dt)

IndexError: too many indices for tensor of dimension 1

In [61]:
plt.plot(tq, measurement)
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 432x288 with 1 Axes>